### Import

Dataset : https://www.kaggle.com/datasets/humansintheloop/semantic-segmentation-of-aerial-imagery

In [1]:
!pip install patchify

In [2]:
import os
import re
from pathlib import Path
import numpy as np
from patchify import patchify
from PIL import Image

### Create patches

In [6]:
def create_folder():
    FOLDERS = ['train','val','test']
    
    for folder in FOLDERS:
        if not os.path.exists(folder):
            folder_imgs = f"{folder}/images"
            folder_msks = f"{folder}/masks"
            os.makedirs(folder_imgs) if not os.path.exists(folder_imgs) else print('folder already exists')
            os.makedirs(folder_msks) if not os.path.exists(folder_msks) else print('folder already exists')

In [7]:
create_folder()

### Create Patches

In [8]:
def create_patches(src, dest_path):
    path_split = os.path.split(src)
    tile_num = re.findall(r'\d+', path_split[0])
    image = Image.open(src)
    image = np.asarray(image)
    if len(image.shape) > 2:
        patches = patchify(image, (320, 320, 3), step = 300)
        file_name_wo_ext = Path(src).stem
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = patches[i, j, 0]
                num = i * patches.shape[1] + j
                patch.save(f"{dest_path}/{file_name_wo_ext}_tile_{tile_number}_patch_{num}.png")
                

In [ ]:
for path_name, _, file_name in os.walk('data'):
    